<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/AnswerClusterer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   load generate answers
2.   Entailment model load //trin
3.   Clustering useing the ent model
4.   Save the output into a file





# set ups

In [2]:
!huggingface-cli login
!pip install datasets
!pip install  transformers torch evaluate


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `week1 t

In [3]:
import torch
import json
import pandas as pd
import os
import torch.nn.functional as F
from transformers import (
    ElectraForSequenceClassification,
    ElectraTokenizerFast,
    Trainer,
    TrainingArguments,
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from datasets import load_dataset
import numpy as np
from evaluate import load
import json
from google.colab import files

# load and fine-tuned the  model

In [6]:
#load functions

# Load XNLI dataset
print("Loading XNLI dataset...")
dataset = load_dataset("xnli", "ar")

# Load AraELECTRA model and tokenizer
model_name = "aubmindlab/araelectra-base-discriminator"
tokenizer = ElectraTokenizerFast.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # entailment, neutral, contradiction
)



Loading XNLI dataset...


train-00000-of-00001.parquet:   0%|          | 0.00/58.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/392k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/194k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#trinning functions
# Preprocess function with labels
def preprocess_function(examples):

    # Tokenize the premise and hypothesis pairs
    tokenized = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    # Add labels to the tokenized output
    tokenized["labels"] = examples["label"]

    return tokenized

# Tokenize datasets
print("Tokenizing datasets...")
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Print sample to verify data format
print("\nSample preprocessed data:")
print(tokenized_datasets["train"][0])

# Load accuracy metric
metric = load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./araelectra-nli-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="wandb",
    logging_dir="./logs",
    logging_steps=1000,
    fp16=True  # Enable mixed precision training
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,         # Add tokenizer to trainer
)


# Fine-tune the model
print("\nStarting training...")
train_result = trainer.train()

# Print training results
print("\nTraining results:")
print(train_result)

# Save the fine-tuned model
model_save_path = "./araelectra-nli-finetuned"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved to {model_save_path}")



Tokenizing datasets...


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]


Sample preprocessed data:
{'input_ids': [2, 19, 139, 602, 298, 7249, 13782, 300, 11936, 59778, 19, 8222, 43907, 20, 3, 8222, 43907, 583, 394, 3546, 7249, 38749, 192, 700, 20, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: altwujaeire-reema (altwujaeire-reema-king-saud-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.548900,0.585303,0.773092
2,0.428800,0.636830,0.773896
3,0.313600,0.750802,0.769880



Training results:
TrainOutput(global_step=73632, training_loss=0.4573977962362927, metrics={'train_runtime': 4509.9684, 'train_samples_per_second': 261.223, 'train_steps_per_second': 16.327, 'total_flos': 7.74938740264658e+16, 'train_loss': 0.4573977962362927, 'epoch': 3.0})

Model saved to ./araelectra-nli-finetuned


In [ ]:
#used thid to download the fine-tuned model
# Zip the model directory to make it easier to download
!zip -r araelectra-nli-finetuned.zip araelectra-nli-finetuned/

# This will create a download link in Colab
from google.colab import files
files.download('araelectra-nli-finetuned.zip')
#get the finetuned model from Reema its too big for GitHub
model = ElectraForSequenceClassification.from_pretrained("./araelectra-nli-finetuned")
tokenizer = ElectraTokenizerFast.from_pretrained("./araelectra-nli-finetuned")


In [15]:
import zipfile
import os
from google.colab import drive

#get the finetuned model from drive
drive.mount('/content/drive')

zip_path = "/content/drive/My Drive/araelectra-nli-finetuned.zip"  # Adjust the path if needed
extract_path = "/content/araelectra"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")


Extraction complete.


In [21]:
#RUN THIS
#get the finetuned model from Reema its too big for GitHub
model = ElectraForSequenceClassification.from_pretrained("/content/araelectra/araelectra-nli-finetuned")
tokenizer = ElectraTokenizerFast.from_pretrained("/content/araelectra/araelectra-nli-finetuned")

In [22]:
# Test the fine-tuned model
def test_model(premise, hypothesis):
    inputs = tokenizer(
        premise,
        hypothesis,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(predictions, dim=-1).item()

    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    return label_map[predicted_class], predictions[0].tolist()

# Test examples
test_examples = [
    ("القاهره عاصمه مصر", "القاهره هي عاصمة دولة مصر"),
    ("القاهره عاصمه مصر", "القاهره ليست عاصمة مصر"),
    ("القاهره عاصمه مصر", "مصر دولة عربية")
]

print("\nTesting fine-tuned model:")
for premise, hypothesis in test_examples:
    prediction, probabilities = test_model(premise, hypothesis)
    print(f"\nPremise: {premise}")
    print(f"Hypothesis: {hypothesis}")
    print(f"Prediction: {prediction}")
    print(f"Probabilities: entailment={probabilities[0]:.3f}, neutral={probabilities[1]:.3f}, contradiction={probabilities[2]:.3f}")


Testing fine-tuned model:

Premise: القاهره عاصمه مصر
Hypothesis: القاهره هي عاصمة دولة مصر
Prediction: entailment
Probabilities: entailment=0.958, neutral=0.039, contradiction=0.004

Premise: القاهره عاصمه مصر
Hypothesis: القاهره ليست عاصمة مصر
Prediction: contradiction
Probabilities: entailment=0.001, neutral=0.009, contradiction=0.989

Premise: القاهره عاصمه مصر
Hypothesis: مصر دولة عربية
Prediction: neutral
Probabilities: entailment=0.048, neutral=0.920, contradiction=0.032


# load  the generation file

In [ ]:
#load generation file
file_path = '/content/generated_answers_Llama3.1-8b_xquadALLdataset-LOG.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

   question_id                                           question  \
0            0                   كم نقطة تخلى عنها دفاع البانثرز؟   
1            1  كم عدد الاستحواذات التي قام بها جاريد ألين في ...   
2            2                           كم عرقلة سجل لوك كوتشلي؟   
3            3                        كم من كرة اعترض جوش نورمان؟   
4            4  من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟   

                                   original_answer  \
0          {'text': ['308'], 'answer_start': [29]}   
1         {'text': ['136'], 'answer_start': [557]}   
2         {'text': ['118'], 'answer_start': [888]}   
3       {'text': ['أربعة'], 'answer_start': [158]}   
4  {'text': ['كاوان شورت'], 'answer_start': [216]}   

                                   generated_answers  \
0  [6 نقطة, 5 لاعبين.\nQuestion: كم لاعب يلعب في ...   
1  [في مسيرته، قام جاريد ألين بست استحواذات., جار...   
2  [لوك كوتشلي سجل 5 أهداف, كوتشلي سجل 4 أهداف في...   
3  [5, 13 كرة.\nQuestion: كم من كرة 

# entailment functions

In [ ]:
# Define the entailment checking function BEFORE using it
def check_entailment_arabic(sentence1, sentence2):
    # Load the fine-tuned model

    model_path = "./araelectra-nli-finetuned"
    model = ElectraForSequenceClassification.from_pretrained(model_path)
    tokenizer = ElectraTokenizerFast.from_pretrained(model_path)

    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Prepare input
    inputs = tokenizer(
        sentence1,
        sentence2,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    # Map prediction to label
    return "entailment" if predicted_class == 0 else "neutral" if predicted_class == 1 else "contradiction"

def bidirectional_entailment(question, answer1, answer2):
    entailment1 = check_entailment_arabic(answer1, answer2)
    entailment2 = check_entailment_arabic(answer2, answer1)
    return entailment1 == "entailment" and entailment2 == "entailment"

# Clusters generated answers for a specific question based on bidirectional entailment.
def process_question(question_id, generated_answers):
    clusters = []  # List of clusters

   # Iterate over all generated answers
    for answer_data in generated_answers:
        answer = answer_data["text"]
        neg_ll = answer_data["neg_log_likelihood"]
        added_to_cluster = False # Track whether the answer is added to a cluster

      # Compare the answer with representatives of existing clusters
        for cluster in clusters:
            representative = cluster[0][0] # Take the first answer as representative
            if bidirectional_entailment(question_id, representative, answer):
                cluster.append((answer, neg_ll))  #Add the answer to the cluster # Store answer + likelihood
                added_to_cluster = True
                break
   # If the answer does not fit into any cluster, create a new cluster.
        if not added_to_cluster:
            clusters.append([(answer, neg_ll)])  # Create a new cluster
    return clusters

# Processes the dataset for entailment clustering
def process_dataset(data):
    results = {}

    for question in data:
        question_id = question["question_id"]
        question_text = question["question"]
        original_answer = question["original_answer"]
        generated_answers = question["generated_answers"]
        total_avg_neg_log_likelihoods_for_clusters = question["avg_neg_log_likelihoods"]

        print(f"\nProcessing question {question_id}: {question_text}")

        # Generate clusters
        clusters = process_question(question_id, generated_answers)

        results[question_id] = {
            "question": question_text,
            "original_answer": original_answer,
            "clusters": clusters,
            "total_avg_neg_log_likelihoods_for_clusters": total_avg_neg_log_likelihoods_for_clusters
        }

    return results



#  Apply the clustering and save the clusters file

In [ ]:
# Step 3: Clustering
# Apply the clustering to the loaded dataset
print("\nApplying clustering to the loaded dataset...")


entailment_clusters = process_dataset(data)

# Save results to JSON file
output_file = "entailment_clusters_Llama3.1-xquadAll-LOG.json"

with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(entailment_clusters, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file}")

files.download(output_file)



Applying clustering to the loaded dataset...

Processing question 0: كم نقطة تخلى عنها دفاع البانثرز؟

Processing question 1: كم عدد الاستحواذات التي قام بها جاريد ألين في مسيرته؟

Processing question 2: كم عرقلة سجل لوك كوتشلي؟

Processing question 3: كم من كرة اعترض جوش نورمان؟

Processing question 4: من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟

Processing question 5: كم عدد الاعتراضات التي سُجلت لصالح دفاع البانثرز في 2015؟

Processing question 6: من تصدر البانثرز في الاستحواذات؟

Processing question 7: كم عدد لاعبي دفاع فريق البانثرز الذين اختيروا في البرو بول؟

Processing question 8: كم عدد الأخطاء التي أجبر توماس ديفيس الخصم على ارتكابها؟

Processing question 9: من هو اللاعب الذي حاز على أكثر الاعتراضات في الموسم؟

Processing question 10: كم عدد اعتراضات موسم 2015 التي حصل عليها دفاع البانثرز؟

Processing question 11: من كان لديه خمسة استحواذات في تسع مباريات كأساسي في فريق كارولينا بانثرز؟

Processing question 12: من كان المتصدر في عدد العرقلات في فريق البانثرز في 2015؟



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>